# Chapter 1

## Exercises 1.1

### 1.1.1 

### 1.1.5

In [2]:
nextfloat(floatmax())

Inf

In [3]:
nextfloat(-Inf64)

-1.7976931348623157e308

Thats interesting. This say that Inf and -Inf sit 'one $\epsilon$ away from the maximum values.